In [ ]:
# This notebook requires holdings.csv file in the same folder. A sample of the CSV file is shown below:
#symbol, quantity, cost
#ADANIGREEN, 100, 1372.54
#ADANIPORTS, 150, 1223


In [ ]:
import requests
import yfinance as yf
import talib as ta
import pandas as pd
import schedule
import time
from datetime import datetime
from backtesting import Backtest, Strategy  # TODO USe Back testing 
from backtesting.lib import crossover

global_cookies = None

In [ ]:
# Check if it's a trading day
def fetch_nse_holidays():
    """
    Fetch the list of NSE holidays from the NSE India website.
    """
    
    baseurl = "https://www.nseindia.com/"
    url = "https://www.nseindia.com/api/holiday-master?type=trading"
    # headers = {
    #     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    #     "Accept": "application/json, text/javascript, */*; q=0.01",
    #     "Accept-Language": "en-US,en;q=0.5",
    #     "Accept-Encoding": "gzip, deflate, br",
    #     "Connection": "keep-alive",
    #     "Referer": "https://www.nseindia.com/",
    #     "X-Requested-With": "XMLHttpRequest"
    # }
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, "
        "like Gecko) "
        "Chrome/80.0.3987.149 Safari/537.36",
        "accept-language": "en,gu;q=0.9,hi;q=0.8",
        "accept-encoding": "gzip, deflate, br",
    }

    try:
        global global_cookies
        
        # print("Fetching NSE holidays...")
        session = requests.Session()
        
        if global_cookies is None:
            request = session.get(baseurl, headers=headers, timeout=10)
            global_cookies = dict(request.cookies)
            
        response = session.get(url, headers=headers, timeout=10, cookies=global_cookies)
        response.raise_for_status()
        holidays = response.json()
        if holidays is not None and "CBM" in holidays:
            return holidays["CBM"]
        return None

    except Exception as e:
        print(f"Error fetching NSE holidays: {e}")
        return []

def is_trading_day():
    """
    Check if today is a trading day by comparing it with the NSE holiday list.
    """

    # print("Checking if today is a trading day...")

    # Fetch NSE holidays
    holidays = fetch_nse_holidays()
    if not holidays:
        print("Unable to fetch holidays. Assuming today is a trading day.")
        return True
    
    # Get today's date in the format used by NSE (e.g., "2023-10-05")
    today = datetime.today().strftime("%Y-%m-%d")
    
    # Check if today is a holiday
    for holiday in holidays:
        if 'tradingDate' in holiday and holiday['tradingDate'] == today:
            print(f"Today ({today}) is a holiday: {holiday['description']}.")
            return False
    
    # Check if today is a weekend
    if datetime.today().weekday() >= 5:  # Saturday (5) or Sunday (6)
        print(f"Today ({today}) is a weekend.")
        return False
    
    # If not a holiday or weekend, it's a trading day
    return True

# Step 10: Check if the market is open now
def is_market_open_now():
    now = datetime.now().time()
    open = datetime.strptime("09:15:00", "%H:%M:%S").time() <= now <= datetime.strptime("15:00:00", "%H:%M:%S").time()
    print (f"Market is open now: {open}")
    return open

# End-of-day summary
def end_of_day_summary():
    global total_buy_orders, total_sell_orders, total_profits, total_investment
    print("\n========== End of Day Summary ==========")
    print(f"Total Buy Orders: {total_buy_orders}")
    print(f"Total investment in Buy Orders: ₹{total_investment:.2f}")
    print(f"Total Sell Orders: {total_sell_orders}")
    print(f"Total Profits from Sell Orders: ₹{total_profits:.2f}")
    print("\n========================================")

In [ ]:
%pip install brotli

# Global variables to track trades and profits
today = None
is_market_open_today = False
virtual_portfolio = {}  # Tracks virtual holdings
total_buy_orders = 0
total_sell_orders = 0
total_profits = 0
total_investment = 0

# Fetch NIFTY stocks list
def fetch_nifty_stocks():

    baseurl = "https://www.nseindia.com/"
    
    # NIFTY50 
    url = f"https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"
    # NIFTY500
    # url = f"https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20500"
    # headers = {
    #     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    #     "Accept": "application/json, text/javascript, */*; q=0.01",
    #     "Accept-Language": "en-US,en;q=0.5",
    #     "Accept-Encoding": "gzip, deflate, br",
    #     "Connection": "keep-alive",
    #     "Referer": "https://www.nseindia.com/",
    #     "X-Requested-With": "XMLHttpRequest"
    # }
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, "
        "like Gecko) "
        "Chrome/80.0.3987.149 Safari/537.36",
        "accept-language": "en,gu;q=0.9,hi;q=0.8",
        "accept-encoding": "gzip, deflate, br",
    }
    
    try:
        global global_cookies
        print("Fetching NIFTY stocks")

        session = requests.Session()
        
        if global_cookies is None:
            request = session.get(baseurl, headers=headers, timeout=10)
            global_cookies = dict(request.cookies)
            
        response = session.get(url, headers=headers, timeout=10, cookies=global_cookies)
        response.raise_for_status()

        data = response.json()
        stocks = data['data']
        stock_list = [stock['symbol'] for stock in stocks]
        print(f"Fetched {len(stock_list)} stocks from .")
        return stock_list
    except Exception as e:
        print(f"****** Error fetching NIFTY stocks: {e}")
        return []

# Step 2: Fetch historical data
def fetch_historical_data(ticker, start_date, end_date):
    # print(f"Fetching historical data for {ticker} from {start_date} to {end_date}.")
    stock = yf.Ticker(f"{ticker}.NS")
    data = stock.history(start=start_date, end=end_date)
    # if data.empty:
    #     print(f"No historical data found for {ticker}.")
    return data

def fetch_currentprice(ticker) -> float:
    # print(f"Fetching current price for {ticker}.")
    stock = yf.Ticker(f"{ticker}.NS")
    price = stock.fast_info.last_price
    if price is None:
        print(f"No current price found for {ticker}.")
    return price


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:

import numpy as np

def check_valid_data(prop, data_dict, ticker) -> bool:
    if prop not in data_dict or data_dict[prop] is None:
        # print(f"*** Insufficient data for {prop} in data_dict for {ticker}.")
        return False
    return True

def rank_stocks_for_swing_buy(historical_data_dict, fundamental_data_dict=None, technical_data_dict=None, derivatives_data_dict=None):
    print(f"Ranking stocks based on technical, fundamental, and derivatives indicators... {len(historical_data_dict)} stocks")

    try:
        ranked_stocks = []
        
        for ticker, historical_data in historical_data_dict.items():
            if historical_data.empty:
                print(f"No historical data available for {ticker}. Skipping.")
                continue

            # print (f"Scoring {ticker}")

            score = 0
            score_reasons = []
            
            # Score based on technical data
            if technical_data_dict is not None and ticker in technical_data_dict:
                tech_data = technical_data_dict[ticker]
                if check_valid_data ('RSI', tech_data, ticker) is not True or check_valid_data ('MACD', tech_data, ticker) is not True or check_valid_data('MACD_signal', tech_data, ticker) is not True \
                    or check_valid_data('SMA_20', tech_data, ticker) is not True or check_valid_data('SMA_50', tech_data, ticker) is not True \
                    or check_valid_data('Lower_Band', tech_data, ticker) is not True or check_valid_data('ADX', tech_data, ticker) is not True or check_valid_data('OBV', tech_data, ticker) is not True :
                    continue

                # Technical Indicators
                if tech_data['RSI'].iloc[-1] < 30:  # Oversold
                    score += 2
                    score_reasons.append("Oversold")
                if tech_data['RSI'].iloc[-1] > 70:  # Overbought (negative score)
                    score -= 1
                if tech_data['MACD'].iloc[-1] > tech_data['MACD_signal'].iloc[-1]:  # Bullish crossover
                    score += 1
                    score_reasons.append("Bullish crossover")
                if historical_data['Close'].iloc[-1] > tech_data['SMA_20'].iloc[-1]:  # Price above SMA 20
                    score += 1
                    score_reasons.append("Price above SMA 20")
                if historical_data['Close'].iloc[-1] > tech_data['SMA_50'].iloc[-1]:  # Price above SMA 50
                    score += 1
                    score_reasons.append("Price above SMA 50")
                if historical_data['Close'].iloc[-1] < tech_data['Lower_Band'].iloc[-1]:  # Price near lower Bollinger Band
                    score += 1
                    score_reasons.append("Price near lower Bollinger Band")
                if tech_data['ADX'].iloc[-1] > 25:  # Strong trend
                    score += 1
                    score_reasons.append("Strong trend")
                if tech_data['OBV'].iloc[-1] > tech_data['OBV'].shift(1).iloc[-1]:  # Increasing volume
                    score += 1
                    score_reasons.append("Increasing volume")
            
            # Score based on fundamental data
            if fundamental_data_dict and ticker in fundamental_data_dict:
                fundamental_data = fundamental_data_dict[ticker]

                trailingPE = fundamental_data['trailingPE']
                dividendYield = fundamental_data['dividendYield']
                if trailingPE is not None and trailingPE < 20:  # Undervalued
                    score += 1
                    score_reasons.append("Undervalued")
                if dividendYield is not None and dividendYield > 2:  # High dividend yield
                    score += 1
                    score_reasons.append("High dividend yield")
            
            # Score based on derivatives data
            if derivatives_data_dict is not None and ticker in derivatives_data_dict:
                derivatives_data = derivatives_data_dict[ticker]
                oi_change = derivatives_data['OI_Change'].iloc[-1]
                pcr = derivatives_data['PCR'].iloc[-1]
                if not np.isnan(oi_change) and oi_change > 0:  # Increasing open interest
                    score += 1
                if not np.isnan(pcr) and pcr < 1:  # Bullish sentiment
                    score += 1
            
            # print (f"   Score for {ticker}: {score}")

            ranked_stocks.append((ticker, score, score_reasons))
        
        # Sort stocks by score in descending order
        ranked_stocks.sort(key=lambda x: x[1], reverse=True)

        # for ticker, score in ranked_stocks[:10]:
        #     print(f"   Ticker: {ticker}, Score: {score}")

        return ranked_stocks
    
    except Exception as e:
        print(f"Error ranking stocks: {e}")
        return []


def fetch_fundamental_data(ticker):
    # print (f"Fetching fundamental data for {ticker}")
    try:
        stock = yf.Ticker(f"{ticker}.NS")  # .NS for NSE stocks
        info = stock.info
        return {
            "trailingPE": info.get('trailingPE', None),
            "dividendYield": info.get('dividendYield', None),
            "debtToEquity": info.get('debtToEquity', None),
            "returnOnEquity": info.get('returnOnEquity', None),
            "targetMeanPrice": info.get('targetMeanPrice', None),
            "regularMarketVolume": info.get('regularMarketVolume', None),
            "averageVolume": info.get('averageVolume', None),
            "recommendationKey": info.get('recommendationKey', None),
            "currentPrice": info.get('currentPrice', None),
        }
    except Exception as e:
        print(f"Error fetching fundamental data for {ticker}: {e}")
        return {}


def calculate_technical_indicators(ticker, historical_data):
    # print (f"Calculating technical data for {ticker}")

    try:
        tech_data = {}
        # Calculate RSI
        tech_data['RSI'] = ta.RSI(historical_data['Close'], timeperiod=14)

        check_valid_data ('RSI', tech_data, ticker)

        # Calculate MACD
        tech_data['MACD'], tech_data['MACD_signal'], tech_data['MACD_hist'] = ta.MACD(historical_data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        
        # Calculate SMA
        tech_data['SMA_20'] = ta.SMA(historical_data['Close'], timeperiod=20)
        tech_data['SMA_50'] = ta.SMA(historical_data['Close'], timeperiod=50)
        
        # Calculate Bollinger Bands
        tech_data['Middle_Band'] = ta.SMA(historical_data['Close'], timeperiod=20)
        tech_data['Upper_Band'] = tech_data['Middle_Band'] + 2 * historical_data['Close'].rolling(window=20).std()
        tech_data['Lower_Band'] = tech_data['Middle_Band'] - 2 * historical_data['Close'].rolling(window=20).std()
        
        # Calculate ADX
        tech_data['ADX'] = ta.ADX(historical_data['High'], historical_data['Low'], historical_data['Close'], timeperiod=14)
        
        # Calculate On-Balance Volume (OBV)
        tech_data['OBV'] = ta.OBV(historical_data['Close'], historical_data['Volume'])

        tech_data['Stochastic_K'], tech_data['Stochastic_D'] = ta.STOCH(historical_data['High'], historical_data['Low'], historical_data['Close'], fastk_period=14, slowk_period=3, slowd_period=3)
        
        return tech_data
    except Exception as e:
        print(f"Error calculating technical indicators for {ticker}: {e}")
        return {}


def fetch_derivatives_data(ticker):
    # TODO - Fetch derivatives data from NSE website
    return None



In [ ]:

# Fetch my portfolio holdings
import csv

# def fetch_my_portfolio_from_dhan():
#     url = "https://api.dhan.co/v2/holdings"
#     headers = {
#         "access-token": f"{DHAN_API_KEY}",
#         "Content-Type": "application/json"
#     }
#     try:
#         response = requests.get(url, headers=headers)
#         if (response.status_code == 500):
#             resp_json = response.json()
#             if 'errorCode' in resp_json and resp_json['errorCode'] == 'DH-1111':
#                 print (f"No holdings available in portfolio")
#                 return []

#         response.raise_for_status()
#         portfolio = response.json()
#         print(f"Fetched {len(portfolio)} holdings from the portfolio.")
#         return portfolio
#     except Exception as e:
#         print(f"Error fetching portfolio: {e}")
#         return []

def fetch_my_portfolio():
    print("Fetching my portfolio holdings...")
    with open('./holdings.csv', mode='r') as file:
        csv_reader = csv.DictReader(file)
        data = [row for row in csv_reader]
        
        for row in data:
            # Convert quantity to integer
            row['quantity'] = int(row['quantity'])
            # Convert current_price to float
            row['cost'] = float(row['cost'])
            # Convert symbol to uppercase
            row['symbol'] = row['symbol'].upper()
        
        for column in data[0].keys():
            print(f"holdings.csv - {column}: {type(data[0][column])}")
    
    return data


# Step 6: Analyze portfolio stocks for sell decisions
def analyze_portfolio_stocks_for_swing_sell():
    print(f"Analyzing portfolio of {len(virtual_portfolio)} stocks for sell decisions...")
    ranked_stocks = []
    for holding in virtual_portfolio:

        ticker = holding['symbol']
        quantity = holding['quantity']
        score = 0
        score_reasons = []

        # print (f"Analyzing {ticker}")

        # Fetch historical data
        historical_data = fetch_historical_data(ticker, "2020-01-01", datetime.now().strftime("%Y-%m-%d"))
        if historical_data is None or historical_data.empty:
            continue

        # Calculate indicators
        technical_data = calculate_technical_indicators(ticker, historical_data)

        if technical_data is not None:
            if check_valid_data ('RSI', technical_data, ticker) is not True or check_valid_data ('MACD', technical_data, ticker) is not True or check_valid_data('MACD_signal', technical_data, ticker) is not True \
                or check_valid_data('SMA_50', technical_data, ticker) is not True:
                continue
            if technical_data['RSI'].iloc[-1] > 70:  # Overbought
                score += 1
                score_reasons.append("RSI > 70 (overbought)")
            if historical_data['Close'].iloc[-1] < technical_data['SMA_50'].iloc[-1]:  # Price below SMA
                score += 1
                score_reasons.append("Price below SMA_50")
            if technical_data['MACD'].iloc[-1] < technical_data['MACD_signal'].iloc[-1]:  # Bearish crossover
                score += 1
                score_reasons.append("MACD bearish crossover")
            if technical_data['Stochastic_K'].iloc[-1] > 80 and technical_data['Stochastic_D'].iloc[-1] > 80:  # Overbought
                score += 1
                score_reasons.append("Stochastic Oscillator > 80 (Overbought)")
            if historical_data['Close'].iloc[-1] > technical_data['Upper_Band'].iloc[-1] > 1:  # Price above upper Bollinger Band
                score += 1
                score_reasons.append("Price above upper Bollinger Band")
            # if technical_data['Volume'].iloc[-1] > technical_data['Volume_SMA_20'].iloc[-1]:  # High volume
            #     score += 1
            #     score_reasons.append("Volume > 20-day SMA Volume")

        fundamental_data = fetch_fundamental_data(ticker)

        if fundamental_data is not None:
            if check_valid_data ('trailingPE', fundamental_data, ticker) and fundamental_data['trailingPE'] > 30:  # High P/E ratio
                score += 1
                score_reasons.append("High trailing P/E ratio")
            if check_valid_data('debtToEquity', fundamental_data, ticker) and fundamental_data['debtToEquity'] > 1:  # High debt to equity ratio
                score += 1
                score_reasons.append("High debt to equity ratio")
            if check_valid_data ('dividendYield', fundamental_data, ticker) and fundamental_data['dividendYield'] < 0.01:  # Low dividend yield
                score += 1
                score_reasons.append("Low dividend yield")
            if check_valid_data('returnOnEquity', fundamental_data, ticker) and fundamental_data['returnOnEquity'] < 0.1:  # Low return on equity
                score += 1
                score_reasons.append("Low return on equity")
            if check_valid_data ('currentPrice', fundamental_data, ticker) and check_valid_data ('targetMeanPrice', fundamental_data, ticker) \
                and fundamental_data['currentPrice'] > fundamental_data['targetMeanPrice']:  # Price above target mean price
                score += 1
                score_reasons.append("Price above target mean price")
            if check_valid_data('regularMarketVolume', fundamental_data, ticker) and check_valid_data('averageVolume', fundamental_data, ticker) \
                and fundamental_data['regularMarketVolume'] > 1.5 * fundamental_data['averageVolume']:  # High volume
                score += 1
                score_reasons.append("High trading volume")
            if check_valid_data('recommendationKey', fundamental_data, ticker) and fundamental_data['recommendationKey'] == 'sell':  # Analyst recommendation
                score += 1
                score_reasons.append("Analyst sell recommendation")            

        derivatives_data = fetch_derivatives_data(ticker)

        if derivatives_data is not None:
            if check_valid_data ('OI_Change', derivatives_data, ticker) is not True or check_valid_data ('PCR', derivatives_data, ticker) is not True or check_valid_data('Implied_Volatility', derivatives_data, ticker) is not True \
                or check_valid_data('Open_Interest', derivatives_data, ticker) is not True or check_valid_data('Open_Interest_SMA_20', derivatives_data, ticker) is not True:
                continue
            if derivatives_data['PCR'].iloc[-1] > 1:  # Bearish sentiment
                score += 1
                score_reasons.append("PCR > 1 (Bearish Sentiment)")
            # if derivatives_data['Implied_Volatility'].iloc[-1] > historical_volatility:  # High implied volatility
            #     score += 1
            #     score_reasons.append("Implied Volatility > Historical Volatility")
            if derivatives_data['Open_Interest'].iloc[-1] < derivatives_data['Open_Interest_SMA_20'].iloc[-1]:  # Decreasing open interest
                score += 1
                score_reasons.append("Open Interest < 20-day SMA Open Interest")

        ranked_stocks.append((ticker, score, score_reasons))

    ranked_stocks.sort(key=lambda x: x[1], reverse=True)
    
    return ranked_stocks


In [6]:

def place_buy_order(symbol, quantity, score_reasons):
    global virtual_portfolio, total_buy_orders, total_sell_orders, total_profits, total_investment
    
    print(f"Placing BUY order: {quantity} shares of {symbol}...")

    # Fetch current price
    data = fetch_currentprice(symbol)
    if data is None:
        print(f"No data found for {symbol}. Skipping order.")
        return
    current_price = data
    
    # Record buy order
    found = False
    for item in virtual_portfolio:
        if item["symbol"] == symbol:
            item["quantity"] += quantity
            found = True
            break
    if not found:
        virtual_portfolio.append({"symbol": symbol, "quantity": quantity})
    
    total_buy_orders += 1
    total_investment += quantity * current_price
    print(f"   BUY ORDER: {quantity} shares of {symbol} at ₹{current_price:.2f}.")
    print(f"       REASON: {score_reasons}")
    
    
def place_sell_order (symbol, quantity, score_reasons):
    global virtual_portfolio, total_buy_orders, total_sell_orders, total_profits, total_investment
    
    print(f"Placing SELL order: {quantity} shares of {symbol}...")

    # Fetch current price
    data = fetch_currentprice(symbol)
    if data is None:
        print(f"No data found for {symbol}. Skipping order.")
        return
    current_price = data
    
    # Record sell order and calculate profit
    found = False
    for item in virtual_portfolio:
        if item["symbol"] == symbol:
            if item["quantity"] >= quantity:
                # Calculate profit (assuming buy price is not tracked here)
                profit = (current_price - 0) * quantity  # Replace 0 with actual buy price if tracked
                total_profits += profit
                item["quantity"] -= quantity
                if item["quantity"] == 0:
                    virtual_portfolio.remove(item)
                total_sell_orders += 1
                print(f"   SELL ORDER: {quantity} shares of {symbol} at ₹{current_price:.2f}.")
                print(f"   Reason: {score_reasons}")
                print(f"   Profit/Loss from this trade: ₹{profit:.2f}.")
            else:
                print(f"Cannot sell {quantity} shares of {symbol}. Not enough holdings.")
            found = True
            break
    if not found:
        print(f"Cannot sell {quantity} shares of {symbol}. Symbol not found in portfolio.")
    


In [ ]:
def rank_stocks():
    # print(f"Entered rank_stocks...")
    
    # Fetch NIFTY stocks
    nifty_stocks = fetch_nifty_stocks()

    print(f"Ranking {len(nifty_stocks)} stocks")
    
    # Fetch historical data, fundamental data and derivatives data for NIFTY stocks
    # historical_data = {ticker: fetch_historical_data(ticker, "2020-01-01", datetime.now().strftime("%Y-%m-%d")) for ticker in nifty_stocks}
    historical_data_dict = {}
    technical_data_dict = {}
    fundamental_data_dict = {}
    for ticker in nifty_stocks:
        data = fetch_historical_data(ticker, "2020-01-01", datetime.now().strftime("%Y-%m-%d"))
        if not data.empty:
            historical_data = {
                ticker: data,
            }
            historical_data_dict.update(historical_data)

            technical_data = {
                ticker: calculate_technical_indicators(ticker, data)
            }
            technical_data_dict.update(technical_data)
            
            fundamental_data = {
                ticker: fetch_fundamental_data(ticker)
            }
            fundamental_data_dict.update(fundamental_data)
            
            derivatives_data = {
                ticker: fetch_derivatives_data(ticker)
            }

    
    # Step 3: rank the stocks for swing buy decisions
    ranked_stocks = rank_stocks_for_swing_buy(historical_data_dict, fundamental_data_dict, technical_data_dict, None) # derivatives_data)

    return ranked_stocks

In [ ]:

# Main algo trading function
def run_algo_trading_once():
    global total_buy_orders, total_sell_orders, total_profits, virtual_portfolio
    
    print(f"Entered run_algo_trading_once...")
    
    ranked_stocks = rank_stocks()
    
    # Place buy orders for top 5 stocks
    for ticker, _, score_reasons in ranked_stocks[:5]:
        place_buy_order(ticker, 1, score_reasons)
    
    # Fetch my portfolio and analyze sell candidates
    virtual_portfolio = fetch_my_portfolio()
    sell_candidates = analyze_portfolio_stocks_for_swing_sell()
    
    # Place sell orders for portfolio stocks
    for ticker, quantity, reason in sell_candidates:
        place_sell_order(ticker, quantity, reason)


In [9]:
def print_report_stockrank_for_swing_buy ():
    ranked_stocks = rank_stocks()
    print ("----- Top 5 stocks to buy ------")
    for ticker, score, score_reasons in ranked_stocks[:5]:
        print(f"BUY {ticker}: score {score} - {score_reasons}")
    print ("--------------------------------")

def print_report_portfolio_for_swing_sell ():
    global virtual_portfolio
     
    virtual_portfolio = fetch_my_portfolio()
    sell_candidates = analyze_portfolio_stocks_for_swing_sell()
    print ("----- Top 10 stocks to sell ------")
    for ticker, score, score_reasons in sell_candidates[:10]:
        print(f"SELL {ticker}: score {score} - {score_reasons}")
    print ("----------------------------------")


In [ ]:
def run_algo_trading_timer():
    global today
    global is_market_open_today

    print ("Checking for algo trading today")
    if today != datetime.today().strftime("%Y-%m-%d"):
        today = datetime.today().strftime("%Y-%m-%d")
        is_market_open_today = is_trading_day()

    date_time = datetime.isoformat(datetime.now())
    
    if is_market_open_today and is_market_open_now():
        print(f"----- Market is open {date_time} - Running algo trading ---")
        run_algo_trading_once ()
    else:
        print(
            f"----- Market is closed {date_time} - Reporting potential buys and sells for the next market day ---"
        )
        print_report_stockrank_for_swing_buy ()
        print_report_portfolio_for_swing_sell ()

In [ ]:

# Schedule the program to run every 60 minutes during market hours
schedule.every(60).minutes.do(run_algo_trading_timer)

# Start the first run now
run_algo_trading_timer()

# Keep the script running
if __name__ == "__main__":
    print("Algo trading program started. Press Ctrl+C to stop.")
    while True:
        schedule.run_pending()
        time.sleep(1)

Checking for algo trading today
Today (2025-07-12) is a weekend.
Entered run_algo_trading_once...
Fetching NIFTY stocks
****** Error fetching NIFTY stocks: 401 Client Error: Unauthorized for url: https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050
Ranking 0 stocks
Ranking stocks based on technical, fundamental, and derivatives indicators... 0 stocks
Fetching my portfolio holdings...
holdings.csv - symbol: <class 'str'>
holdings.csv - quantity: <class 'int'>
holdings.csv - cost: <class 'float'>
Analyzing portfolio of 15 stocks for sell decisions...
Placing SELL order: 5 shares of EXIDEIND...
   SELL ORDER: 5 shares of EXIDEIND at ₹380.65.
   Reason: ['Price below SMA_50', 'MACD bearish crossover', 'High trailing P/E ratio', 'High debt to equity ratio', 'Low return on equity']
   Profit/Loss from this trade: ₹1903.25.
Placing SELL order: 4 shares of ETERNAL...
   SELL ORDER: 4 shares of ETERNAL at ₹263.35.
   Reason: ['MACD bearish crossover', 'High trailing P/E ratio', 'Hi

KeyboardInterrupt: 